<div style="text-align: right;">
<a target="_blank" href="https://colab.research.google.com/github/hkaragah/hkaragah.github.io/blob/main/structure/opensees/opensees_steel_materials.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<div style="font-size: 0.8em; color: #555;">By Hossein Karagah</div>
<div style="font-size: 0.8em; color: #555;">© Copyright 2025 GNU GENERAL PUBLIC LICENSE.</div>
</div>

# OpenSees: Two-Story MRF Analysis

In [1]:
from __future__ import annotations
from dataclasses import dataclass
import os
import sys
sys.path.append(os.path.abspath("../.."))
from assets.modules.shapes import AISC_WSection
from assets.modules.materials import SteelSectionCategories, ASTMSteel
import openseespy.opensees as ops
from math import prod

In [12]:
# Define steel material
E = 29000 # ksi
Fy = 50.0 # ksi
cat = SteelSectionCategories.HOT_ROLLED
A992 = ASTMSteel('A992', cat, Fy, E)

# Define beam and column sections
bm = AISC_WSection('W10X26')
col = AISC_WSection('W8X48')

In [15]:
# Compute Bilin material parameters

# Regression coefficients from Lignos & Krawinkler (2011)
A = {
    'd<21': {
#                            h/tw    bf/2tf  Lb/ry  L/d    c1*d    c2*Fy
        'theta_p':  [0.0865, -0.365, -0.140, 0.000, 0.340, -0.721, -0.230],
        'theta_pc': [5.6300, -0.565, -0.800, 0.000, 0.000, -0.280, -0.430],
        'lambda':   [495.00, -1.340, -0.595, 0.000, 0.000,  0.000, -0.360]
    },
    'd>=21': {
        'theta_p':  [0.3180, -0.550, -0.345, -0.023, 0.090, -0.330, -0.130],
        'theta_pc': [7.5000, -0.610, -0.710, -0.110, 0.000, -0.161, -0.320],
        'lambda':   [536.00, -1.260, -0.525, -0.130, 0.000,  0.000, -0.291]
    },
    'rbs': {
        'theta_p':  [0.1900, -0.314, -0.100, -0.185, 0.113, -0.760, -0.070],
        'theta_pc': [9.5200, -0.513, -0.863, -0.108, 0.000,  0.000, -0.360],
        'lambda':   [585.00, -1.140, -0.632, -0.205, 0.000,  0.000, -0.391]
    }
}



def _nonrbs_LK(sec: AISC_WSection, is_rbs: bool,mat: ASTMSteel, L: float, Lb: float | None = None) -> dict:
    """
    Lignos–Krawinkler median regressions RBS or WUF-B connections:
    Returns theta_p, theta_pc, Lambda, theta_u.
    """
    Fy = mat.fy
    d = sec.d
    ry = sec.ry
    htw = sec.htw
    bf2tf = sec.bf2tf
    if not Lb:
        Lb = 2500 / mat.fy
    Lbry = Lb / ry
    Ld = L / d
    c1 = c2 = 1.0  # inch/ksi units

    if d < 21.0:
        m = {'theta_p': 0.0865, 'theta_pc': 5.63, 'lambda': 495}
        p = {
            'theta_p':[-0.365, -0.140, 0.000, 0.340, -0.721, -0.230],
            'theta_pc':[-0.565, -0.800, 0.000, -0.280, -0.430],
            'lambda':[-1.34, -0.595, 0.000, -0.360]
        }
    else:
        m = {'theta_p': 0.318, 'theta_pc': 4.03, 'lambda': 536}
        p = {
            'theta_p':[-0.550, -0.345, -0.0230, 0.090, -0.330, -0.130],
            'theta_pc':[-0.610, -0.710, -0.110, -0.161, -0.320],
            'lambda':[-1.26, -0.525, -0.130, -0.291]
        }
        
    if is_rbs:
        a = A['rbs']
    else:
        a = A['d<21'] if d < 21.0 else A['d>=21']
    
    KL_params ={}
    for key in a.keys(): # keys: 'theta_p', 'theta_pc', 'lambda'
        KL_params[key] = prod([
            a[key][0],
            htw**(a[key][1]),
            bf2tf**(a[key][2]),
            Lbry**(a[key][3]),
            Ld**(a[key][4]),
            (c1*d/533)**(a[key][5]),
            (c2*Fy/355)**(a[key][6])
            ])
        


    # Ultimate rotation capacity (theta_u)
    # See Lignos & Krawinkler, 2011, Ultimate Rotation Capacity Section
    KL_params['theta_u']  = 0.06 if is_rbs else 0.05 # radians
    
    return KL_params


def _K0_from_hinge(sec: AISC_WSection, mat: ASTMSteel, My: float, Lp: float | None):
    """
    K0 = My / theta_y, with theta_y ≈ (My/(E*Ig)) * Lp.  Default Lp = 0.8*d.
    """
    E, Ig = mat.E, sec.Ix
    Lp = 0.8*sec.d if Lp is None else Lp
    phi_y   = My/(E*Ig)      # 1/in
    theta_y = phi_y*Lp       # rad
    return My/theta_y        # kip-in/rad


def compute_bilin_params_wufb(
    sec: AISC_WSection,
    is_rbs: bool,
    mat: ASTMSteel,
    L: float,                   # in, shear span (hinge to inflection)
    Lb: float | None = None,    # in, unbraced length (default 2500/Fy)
    Lp: float | None = None,    # in, hinge length (default 0.8d)
    as_pos: float = 0.02,       # post-yield hardening ratio (+)
    as_neg: float = 0.02,       # post-yield hardening ratio (–)
    Res_pos: float = 0.20,      # residual strength ratio (+)
    Res_neg: float = 0.20,      # residual strength ratio (–)
    c_S: float = 1.0, c_C: float = 1.0, c_K: float = 1.0,
    D_pos: float = 1.0, D_neg: float = 1.0,
    nFactor: float = 0.0        # elastic stiffness amplifier (optional)
):
    """
    Returns:
      params: dict of Bilin arguments
      cmd:    ready-to-paste OpenSeesPy 'uniaxialMaterial("Bilin", ...)' line
    """
    # Effective yield moment (take sign in command):
    My = mat.Ry * mat.fy * sec.Zx  # kip-in

    # Lignos–Krawinkler non-RBS (WUF-B) medians
    KL_params = _nonrbs_LK(sec, mat, L)

    # Elastic stiffness
    K0 = _K0_from_hinge(sec, mat, My, Lp)

    # Bilin expects Lamda_A although accel-reload deterioration is not active for Bilinear;
    # pass a huge number to effectively disable it.
    Lamda_S = Lamda_C = Lamda_K = KL_params['lambda']
    Lamda_A = 1.0e16

    params = dict(
        K0=K0,
        as_Plus=as_pos, as_Neg=as_neg,
        My_Plus=+My,   My_Neg=-My,
        Lamda_S=Lamda_S, Lamda_C=Lamda_C, Lamda_A=Lamda_A, Lamda_K=Lamda_K,
        c_S=c_S, c_C=c_C, c_A=1.0, c_K=c_K,
        theta_p_Plus=KL_params['theta_p'], theta_p_Neg=KL_params['theta_p'],
        theta_pc_Plus=KL_params['theta_pc'], theta_pc_Neg=KL_params['theta_pc'],
        Res_Pos=Res_pos, Res_Neg=Res_neg,
        theta_u_Plus=KL_params['theta_u'], theta_u_Neg=KL_params['theta_u'],
        D_Plus=D_pos, D_Neg=D_neg,
        nFactor=nFactor
    )

    cmd = (
        "uniaxialMaterial('Bilin', matTag, "
        f"{params['K0']:.6g}, {params['as_Plus']:.6g}, {params['as_Neg']:.6g}, "
        f"{params['My_Plus']:.6g}, {params['My_Neg']:.6g}, "
        f"{params['Lamda_S']:.6g}, {params['Lamda_C']:.6g}, {params['Lamda_A']:.6g}, {params['Lamda_K']:.6g}, "
        f"{params['c_S']:.6g}, {params['c_C']:.6g}, {params['c_A']:.6g}, {params['c_K']:.6g}, "
        f"{params['theta_p_Plus']:.6g}, {params['theta_p_Neg']:.6g}, {params['theta_pc_Plus']:.6g}, {params['theta_pc_Neg']:.6g}, "
        f"{params['Res_Pos']:.6g}, {params['Res_Neg']:.6g}, "
        f"{params['theta_u_Plus']:.6g}, {params['theta_u_Neg']:.6g}, "
        f"{params['D_Plus']:.6g}, {params['D_Neg']:.6g}"
        f"{'' if params['nFactor']==0 else ', ' + str(params['nFactor'])}"  # optional
        ")"
    )
    return params, cmd

In [16]:
params, cmd = compute_bilin_params_wufb(
    sec=bm,
    mat=A992,
    L=0.8*20*12
)

TypeError: compute_bilin_params_wufb() missing 1 required positional argument: 'is_rbs'

In [ ]:
###############
# Reset Model #
###############

ops.wipe()



################
# Create Model #
################

# 2 dimensions, 3 DOF per node
ops.model('basic', '-ndm', 2, '-ndf', 3)

# Define variables
story_height = 12 * 12 # inches
span = 20 * 12 # inches
nStoreis = 3
nBays = 1
nodeTags = {}



################
# Define Nodes #
################

for story in range(nStoreis + 1):
    for bay in range(nBays + 1):
        
        nodeTag = 10 * story + bay + 1
        x = bay * span
        y = story * story_height
        ops.node(nodeTag, x, y)
        nodeTags[(story, bay)] = nodeTag
        
        # Define extra node in the same coordinate for beam plastic hinges
        if story != 0:
            nodeTagPH = (10 * story + bay + 1) * 10
            ops.node(nodeTagPH, x, y)
            nodeTags[(story, bay, 'PH')] = nodeTagPH
            ops.equalDOF(nodeTagPH, nodeTag, 1, 2) # Tie translational DOFs (UX an UY)
            
        # Fix base nodes
        if story == 0:
            ops.fix(nodeTag, 1, 1, 1)



####################
# Define Materials #
####################

b = 0.01 # initial hardening ratio
matTags = [1, 2, 3]

# Nonlinear steel material
ops.uniaxialMaterial('Steel01', matTags[0], A992.fy, A992.E, b)

# elastic steel material
ops.uniaxialMaterial('Elastic', matTags[1], E)

# Define bilin material for plastic hinges
ops.uniaxialMaterial('IMKBilin', matTags[2], 
    params['K0'], params['as_Plus'], params['as_Neg'],
    params['My_Plus'], params['My_Neg'],
    params['Lamda_S'], params['Lamda_C'], params['Lamda_A'], params['Lamda_K'],
    params['c_S'], params['c_C'], params['c_A'], params['c_K'],
    params['theta_p_Plus'], params['theta_p_Neg'],
    params['theta_pc_Plus'], params['theta_pc_Neg'],
    params['Res_Pos'], params['Res_Neg'],
    params['theta_u_Plus'], params['theta_u_Neg'],
    params['D_Plus'], params['D_Neg']
)


coordTransf = "PDelta"  # Linear, PDelta, Corotational
transfTag = 1
ops.geomTransf(coordTransf, transfTag)



###################
# Define Sections #
###################

colSecTag = 1                         
Nfw = 8 # number of fibers in web
Nff = 8 # number of fibers in flange
ops.section('WFSection2d', colSecTag, matTags[0], col.d, col.tw, col.bf, col.tf, Nfw, Nff)

bmSecTag = 2
ops.section('Elastic', bmSecTag, A992.E, bm.A, bm.Ix)



###################
# Define Elements #
###################

nIntPoints = 4 # max = 10
for story in range(nStoreis + 1):
    
    # Define beam element (elastic beam-column element)
    eleTag = 100 * (story + 1) + 11
    nodeTags = (nodeTags[(story + 1, 0, 'PH')], nodeTags[(story + 1, 1, 'PH')])
    ops.element('elasticBeamColumn', eleTag, *nodeTags, bmSecTag, transfTag)
        
    for bay in range(nBays + 1):
        
        # Define column element (nonlinear beam-column element)
        eleTag = 100 * (story + 1) + (bay + 1)
        nodeTags = (nodeTags[(story, bay)], nodeTags[(story + 1, bay)])
        ops.element('nonlinearBeamColumn', eleTag, *nodeTags, nIntPoints, colSecTag, transfTag)
                
        # Define beam plastic hinges (zeoro-length element with Bilin material)
        eleTag = 1000 * (story + 1) + 110 + bay
        nodeTags = (nodeTags[(story + 1, bay + 1)], nodeTags[(story + 1, bay + 1, 'PH')])
        ops.element('zeroLength', eleTag, *nodeTags, '-mat', matTags[2], '-dir', 6)



########################
# Define Gravity Loads #
########################

tsTag = 1
ops.timeSeries('Linear', tsTag, '-factor', 1.0)

DL = 1.4 # klf, distributed load
CL = 11.5 # kips, concentrated load

patternTag = 1
# Define loads in gravity direction (-UY)
ops.pattern('Plain', patternTag, tsTag, {
    ops.load(11, 0.0, -CL, 0.0), 
    ops.load(21, 0.0, -CL, 0.0),
    ops.load(31, 0.0, -CL, 0.0),
    ops.eleLoad('-ele', *[111, 211, 311], '-type', '-beamUniform', 0.0, -DL, 0.0)
})


#################
# Define Masses #
#################

# Define masses in the lateral direction only (UX)
ops.mass(11, 165.0, 0.0, 0.0)
ops.mass(21, 165.0, 0.0, 0.0)
ops.mass(31, 165.0, 0.0, 0.0

intTag = 520
ops.beamIntegration('Lobatto', intTag, secTag, intPoints) # 'Lobatto', 'Legendre', 'NewtonCotes'


IMK with Bilinear Response - Code by AE_KI (Nov22)
